## 📝 Homework Solutions

### ✅ Q1: Standard deviation of predictions  
**Answer:** `6.25`

---

### ✅ Q2: Size of the saved model (`model.bin`)  
**Answer:** `66 MB`

---

### ✅ Q3: Convert notebook to script  

jupyter nbconvert --to script starter.ipynb

---

### ✅ Q4: First hash for scikit-learn in Pipfile.lock

Answer:
sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c

---

### ✅ Q5: Mean predicted duration (April 2023)

Answer: 14.29

---

### ✅ Q6: Mean predicted duration from Docker container (May 2023)

Answer: 0.19

In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.9.21


In [3]:
import pickle
import pandas as pd

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [8]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [15]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [16]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

# Q1. Standard deviation

In [17]:
import numpy as np
np.std(y_pred)

6.247488852238703

# Q2. Preparing the output

In [18]:
# Create ride_id
df['ride_id'] = f'2023/03_' + df.index.astype('str')

# Create output DataFrame
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

# Save to parquet
df_result.to_parquet(
    'homework_output.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [19]:
ls -lh homework_output.parquet

-rw-rw-rw- 1 codespace codespace 66M Jun 12 08:03 homework_output.parquet
